In [1]:
from relaylab import equipment as eqp, signals as sg, record, characteristics, functions as func
from relaylab.signals import color
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Анализ работы дифференциальной защиты трансформатора

### Ввод параметров электрооборудования

In [2]:
tr = eqp.Transformer(16000, 115, 11, Usc=0.105, scheme_high='Y', scheme_low='D', group=11)
ct_vn = eqp.CT(500, 5, Knom=10, Snom=10, cos_nom=0.8,  r2=0.06, x2=0.02, Sload=5, cos_load=0.8)
ct_nn = eqp.CT(1500, 5, Knom=15, Snom=20, cos_nom=0.8,  r2=0.15, x2=0, Sload=5, cos_load=0.8)
dif_har = characteristics.DifRelay(start=0.2, slope_st1=0.5, slope1=0.1, slope_st2=1.5, slope2=0.3)

### Загрузка осциллограммы

In [3]:
rec = record.examples.transformer_short_circuit_out_zone
print(rec)

Наименование осциллограммы: C:\python\RelayLab\relaylab\records\transformer_short_circuit_out_zone.cfg
Ревизия стандарта: 2000
Объект установки устройства: 
Наименование устройства: КИТ-Р-А3-ДЗТ-01_00 версия 1701159520
Начало записи: 2023-11-29 14:22:18.353000
Длительность записи: 2.10, c
Количество аналоговых каналов: 7
Количество дискретных каналов: 79


In [4]:
IA1, IB1, IC1 = rec.IA_VN, rec.IV_VN, rec.IS_VN
IA2, IB2, IC2 = rec.IA_NN, rec.IV_NN, rec.IS_NN

### Расчет дифференциальных токов

In [5]:
IdifA, IdifB, IdifC, IbiasA, IbiasB, IbiasC = func.dif_two_winding_transformer(IA1, IB1, IC1,IA2, IB2, IC2,
                                                                             transformer=tr, CT_high=ct_vn, CT_low=ct_nn)
har_points = dif_har.get_points()

In [11]:
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], specs=[[{"type": "xy"}, {"type": "xy"}]])

fig.add_trace(go.Scatter(IdifA.dft_abs().plt_data, line=dict(color=color.orange)),row=1, col=1)
fig.add_trace(go.Scatter(IbiasA.plt_data, line=dict(color=color.blue)),row=1, col=1)
fig.add_trace(go.Scatter(name='Iдиф', x=IbiasA.val, y=IdifA.dft_abs().val,
                         line=dict(color=color.orange)), row=1, col=2)
fig.add_trace(go.Scatter(name='Характеристика', x=har_points[0], y=har_points[1], 
                         mode='lines', line=dict(color=color.grey)),
              row=1, col=2)

fig.add_trace(go.Scatter(name='Iдиф', x=np.array([0]), y=np.array([0]),
                         mode='markers', marker=dict(color=color.orange, size=10)),
              row=1, col=2)

fig.update_layout(template='plotly_white', height=400,
                  shapes=[dict(type= 'line', line= dict(color=color.red_deep),
                               yref= 'paper', y0= 0, y1= 1,
                               xref= 'x', x0= 0.05, x1= 0.05)],
                  xaxis2=dict(range=[0, max(IbiasA.val)]), yaxis2=dict(range=[0, max(IbiasA.val)]),
                 )

# Интерактивность графика
fig_widget = go.FigureWidget(fig)
scatter1 = fig_widget.data[0]
scatter2 = fig_widget.data[1]
point = fig_widget.data[-1]

def update_point(trace, points, selector):
    if len(points.point_inds) != 0:
        ind = points.point_inds[0]
        fig_widget.update_layout(shapes=[dict(x0= points.xs[0], x1= points.xs[0])])
        point.x = np.array(IbiasA.val[ind])
        point.y = np.array(IdifA.dft_abs().val[ind])

scatter1.on_click(update_point)
scatter2.on_click(update_point)
fig_widget

FigureWidget({
    'data': [{'line': {'color': '#F3A505'},
              'name': 'abs(DFT IдифA)',
              'type': 'scatter',
              'uid': 'fd174358-6960-4385-abe9-ca2b6b7ae3e0',
              'x': array([0.00000000e+00, 4.16666667e-04, 8.33333333e-04, ..., 2.10375000e+00,
                          2.10416667e+00, 2.10458333e+00]),
              'xaxis': 'x',
              'y': array([0.        , 0.        , 0.        , ..., 0.00195595, 0.00258074,
                          0.00258074]),
              'yaxis': 'y'},
             {'line': {'color': '#0095B6'},
              'name': 'IтормA',
              'type': 'scatter',
              'uid': 'ecdc0a7d-e335-48b8-a78d-21bc60b6c613',
              'x': array([0.00000000e+00, 4.16666667e-04, 8.33333333e-04, ..., 2.10375000e+00,
                          2.10416667e+00, 2.10458333e+00]),
              'xaxis': 'x',
              'y': array([0.        , 0.        , 0.        , ..., 0.0011024 , 0.00138495,
                    